In [1]:
import os
import shutil
from time import sleep
from pprint import pprint

import tensorflow as tf
import numpy as np

from batchflow.models.tf import EfficientNetB0

W0911 15:46:28.086140 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/losses/dice.py:6: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0911 15:46:28.087600 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/losses/dice.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W0911 15:46:28.089424 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:22: The name tf.losses.mean_squared_error is deprecated. Please use tf.compat.v1.losses.mean_squared_error instead.

W0911 15:46:28.090591 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:23: The name tf.losses.sigmoid_cross_entropy is deprecated. Please 

In [2]:
from batchflow import Pipeline, B, V, C, D, ImagesBatch
from batchflow.opensets import CIFAR10
from batchflow.research import Option, Research, Results

In [3]:
cifar = CIFAR10(batch_class=ImagesBatch, bar=True)


100%|██████████| 6/6 [00:05<00:00,  1.23s/it]


In [4]:
new_res = EfficientNetB0.resolution
im_shape = (new_res, new_res, 3)

model_config = {
    'inputs/images/shape': im_shape,
    'inputs/labels/classes': D('num_classes'),
    'initial_block/inputs': 'images',
    'loss': 'ce'
}

train_ppl = (cifar.train.p
                  .resize((new_res, new_res))
                  .init_variable('loss_history', default=[])
                  .init_model('dynamic', EfficientNetB0, 'mnist_model', config=model_config)
                  .to_array()
                  .train_model('mnist_model', fetches='loss', images=B('images'), labels=B('labels'), 
                               save_to=V('loss_history', mode='a'))
                  .run_later(64, shuffle=True, n_epochs=1, drop_last=True))

In [ ]:
train_ppl1 = train_ppl.run(bar=True)

  0%|          | 0/781 [00:00<?, ?it/s]W0911 15:46:34.600257 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:319: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0911 15:46:34.602431 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:321: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0911 15:46:34.611655 140350399465280 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:380: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W0911 15:46:34.620984 140350399465280 deprecation.py:323] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/layers/conv.py:26: conv2d (from tensorflow.python.layers.convolutio

In [ ]:
from matplotlib import pyplot as plt
plt.plot(train_pipeline.get_variable('loss_history'))

In [ ]:
boom

In [ ]:
model_config = {
    'inputs/images/shape': B('image_shape'),
    'inputs/labels/classes': D('num_classes'),
    'initial_block/inputs': 'images',
    
    'common/width_factor': C('width_factor'),
    'common/depth_factor': C('depth_factor'),
    
    'loss': 'crossentropy',
    'optimizer': 'Adam', 
}

train_ppl = (dataset.train.p
                  .init_variable('loss') 
                  .init_model('dynamic', ScalableModel, 'mnist_model', config=model_config)
                  .to_array()
                  .train_model('mnist_model', fetches='loss',
                               images=B('images'), labels=B('labels'), 
                               save_to=V('loss'))
             .run_later(BATCH_SIZE, shuffle=True, n_epochs=None))

test_ppl = (dataset.test.p
                 .import_model('mnist_model', C('import_from'))
                 .init_variable('loss')
                 .init_variable('predictions')
                 .init_variable('metrics')
                 .to_array()
                 .predict_model('mnist_model', fetches=['loss', 'predictions'],
                                images=B('images'), labels=B('labels'), 
                                save_to=[V('loss'), V('predictions')])
                 .gather_metrics('class', targets=B('labels'), predictions=V('predictions'),
                                 fmt='logits', axis=-1, save_to=V('metrics'))
            .run_later(BATCH_SIZE, shuffle=True, n_epochs=1))


In [ ]:
op1 = Option('width_factor', [1, 1.5, 2])
op2 = Option('depth_factor', [1, 2])
grid = op1 * op2

list(grid.gen_configs())

In [ ]:
research = (Research()
            .add_pipeline(train_ppl, variables='loss', name='train')
            .add_pipeline(test_ppl, variables='loss', name='test', execute=10,
                          run=True, import_from='train')
            .get_metrics(pipeline='test', metrics_var='metrics', metrics_name='accuracy',
                         returns='accuracy', execute=10)
            .add_grid(grid))

In [ ]:
!rm -r scalable_research

In [ ]:
NUM_REPEATS = 3
NUM_ITERS = 100
research_name = 'scalable_research'

research.run(n_reps=NUM_REPEATS, n_iters=NUM_ITERS, name=research_name, bar=True)


In [ ]:
res = research.load_results(use_alias=True)
res.info()

In [ ]:
from batchflow.utils import plot_results_by_config

plot_results_by_config(res, (('train', 'loss'), ('test', 'loss'), ('test_metrics', 'accuracy')))

In [ ]:
model_config = {
    'inputs/images/shape': B('image_shape'),
    'inputs/labels/classes': D('num_classes'),
    'initial_block/inputs': 'images',
    
    'common/width_factor': 3,
    'common/depth_factor': 2,
    
    'loss': 'crossentropy',
    'optimizer': 'Adam', #('Momentum', {'use_nesterov': True, 'learning_rate': 0.01, 'momentum': 0.5}),
#     'output': ['proba']
}

train_template = (Pipeline()
                  .init_variable('loss_history', default=[])
                  .init_model('dynamic', ScalableModel, 'mnist_model', config=model_config)
                  .to_array()
                  .train_model('mnist_model', fetches='loss', images=B('images'), labels=B('labels'), 
                               save_to=V('loss_history', mode='a'))
                  .run_later(64, shuffle=True, n_epochs=1, drop_last=True, bar=True))

In [ ]:
train_pipeline = (train_template << dataset.train).run()

plt.plot(train_pipeline.get_variable('loss_history'))
